In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/content/drive/MyDrive/Research_dataset/DevanagariHandwrittenCharacterDataset/Train'

train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 32)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.08, 0.08)
    ),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

final_train = datasets.ImageFolder(train_dir, transform=train_transform)
loader = torch.utils.data.DataLoader(
    dataset=final_train, batch_size=32, shuffle=True)


In [ ]:
class AE(nn.Module):
    def __init__(self):
        super(AE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(32 * 32, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 36),
            nn.ReLU(),
            nn.Linear(36, 18),
            nn.ReLU(),
            nn.Linear(18, 9)  # latent space compact represnetation of an image 162
        )
        self.decoder = nn.Sequential(
            nn.Linear(9, 18),
            nn.ReLU(),
            nn.Linear(18, 36),
            nn.ReLU(),
            nn.Linear(36, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 32 * 32),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = AE()
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-8)

In [ ]:
from tqdm.auto import tqdm

epochs = 20
outputs = []
losses = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in tqdm(range(epochs), desc="Training Epochs"):
    for images, _ in tqdm(loader, desc=f"Epoch {epoch+1}", leave=False):
        images = images.view(-1, 32 * 32).to(device)

        reconstructed = model(images)
        loss = loss_function(reconstructed, images)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

    outputs.append((epoch, images, reconstructed))
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

plt.style.use('fivethirtyeight')
plt.figure(figsize=(8, 5))
plt.plot(losses, label='Loss')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# comparing the original from dataset and model reconstruction

model.eval()
dataiter = iter(loader)
images, _ = next(dataiter)

images = images.view(-1, 32 * 32).to(device)
reconstructed = model(images)

fig, axes = plt.subplots(nrows=2, ncols=10, figsize=(10, 4)) # Increased height for titles

for i in range(10):
    axes[0, i].imshow(images[i].cpu().detach().numpy().reshape(32, 32), cmap='gray')
    axes[0, i].axis('off')
    if i == 0: # Add title only to the first subplot in the row
        axes[0, i].set_title('Original Images')

    axes[1, i].imshow(reconstructed[i].cpu().detach().numpy().reshape(32, 32), cmap='gray')
    axes[1, i].axis('off')
    if i == 0: # Add title only to the first subplot in the row
        axes[1, i].set_title('Reconstructed Images')

plt.tight_layout()
plt.show()